**OBLIGATORIO FINAL G448 - Data Wrangling**
<br>**INTEGRANTES:**
<br>
+ Theo Pereira, 5703834-3, theopereira2001@gmail.com
+ Carlos Píriz, 4298784-4, piriz.carlos@gmail.com
+ Gabriel	Better,3138871-2, gabrielbetter@gmail.com

In [947]:
### Importacion de librerias ###

# Sistema
import os

# Tratamiento de datos
import pandas as pd
import numpy as np
import datetime as dt
import math

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Visualizacion 
import seaborn as sns
import matplotlib as mpl
from matplotlib import ticker
from matplotlib.ticker import FuncFormatter
from babel.numbers import format_currency
import matplotlib.pyplot as plt
%matplotlib inline

**CONSIGNAS**
<br>
El objetivo general es realizar un análisis exploratorio de los anuncios inmobiliarios de
algunas provincias de la Patagonia Argentina reportados por el portal Airbnb.
<br>
Los datos están conformados por cuatro archivos csv:
+ Neuquen.csv
+ Rio_Negro.csv
+ Chubut.csv
+ Tierra_del_Fuego.csv

**<h1>1. Carga de datos y armado del dataset</h1>**

In [948]:
# Carga de archivos de datos de chubut
df_chubut = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Chubut.csv')
df_chubut["provincia"]="CHUBUT"
df_chubut.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
77,844132854151774786,Estepa,entire_home,Nuevo,Comodoro Rivadavia,-45.869540,-67.497530,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$71 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
181,637535602235296049,Etxea II Departamento amoblado,entire_home,NaN,Trelew,-43.259900,-65.296620,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
196,54214936,TINY LOFT 36 m2 NUEVO (centro de Esquel),entire_home,"4,72 (43)",Esquel,-42.912890,-71.314110,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
164,916099436446686089,Casa Los Rombos.,entire_home,Nuevo,Trevelin,-43.087152,-71.466290,"['2 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$24 USD,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
236,602279361080194562,Cañadon bonito - Cabaña Cipres,entire_home,NaN,Cushamen,-42.111510,-71.441810,"['No hay evaluaciones', '7 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$93 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
188,34181221,"Hermoso,luminoso y muy bien ubicado departamento*",entire_home,"4,93 (58)",Puerto Madryn,-42.771760,-65.030600,"['1 cama', '1 dormitorio', '1 baño', '1 cama k...",2023-07-10,2023-07-15,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
81,734738900808999694,Hermoso depto cerca del mar,entire_home,"4,96 (27)",Puerto Madryn,-42.756011,-65.037988,"['1 cama', '1 dormitorio', '1 baño', 'Califica...",2023-07-16,2023-07-21,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
210,630333448514430277,Hermoso bed and breakfast frente al Atlántico,private_room,NaN,Puerto Madryn,-42.748570,-65.038810,"['No hay evaluaciones', '6 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$67 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
93,709585936691929156,"Phira rojo, habitación con vista al mar",private_room,"4,73 (11)",Puerto Pirámides,-42.570180,-64.278739,"['1 cama', '1 dormitorio', '1 baño privado']",2023-08-28,2023-09-02,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
82,573284010642445099,Les Anges,entire_home,"4,91 (11)",Trelew,-43.272508,-65.253550,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-20,2023-08-25,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [949]:
# Carga de archivos de datos de Neuquen
df_neuquen = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Neuquen.csv')

df_neuquen["provincia"] = "NEUQUEN"

df_neuquen.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
172,862987897159647375,Departamento Amoblado Neuquén,entire_home,NaN,Neuquén,-38.948100,-68.061830,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,$47 USD,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
49,804960727499903237,Babel Soft - Las Violetas,entire_home,"3,33 (3)",Neuquén,-38.943190,-68.050970,"['2 camas', '1 dormitorio', '1 baño', 'Estacio...",2023-07-03,2023-07-08,NaN,$30 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
160,715234401046472374,Hermoso dpto. a una cuadra de Av. Argentina,entire_home,NaN,Neuquén,-38.952920,-68.057710,"['No hay evaluaciones', '2 camas', '1 dormitor...",2023-07-01,2023-07-06,NaN,$45 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
207,35734921,4 - Pura tranquilidad en el corazón de Neuquén,private_room,"3,75 (4)",Neuquén,-38.953170,-68.105900,"['3 camas', '2 dormitorios', '1 cama doble', '...",2023-07-01,2023-07-06,NaN,$55 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
74,685624345010270874,"Casa dúplex , con patio",entire_home,NaN,Neuquén,-38.948400,-68.113740,"['No hay evaluaciones', '3 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$51 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
174,678351386809115205,Hermoso Departamento en Cipolletti,entire_home,NaN,Cipolletti,-38.936570,-67.992450,"['2 camas', '1 dormitorio', '1 baño', 'Wifi']",2023-12-29,2024-01-03,NaN,$40 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
171,22901633,"Mi espacio, tu espacio!",entire_home,"4,39 (18)",Neuquén,-38.956720,-68.082640,"['3 camas', '1 dormitorio', '1 ,5 baños', '1 c...",2023-07-01,2023-07-06,NaN,$58 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
216,37603595,5 - Pura tranquilidad en el corazón de Neuquén,shared_room,NaN,Confluencia Department,-38.951840,-68.058100,"['2 camas', '2 dormitorios', '2 baños comparti...",2023-07-01,2023-07-06,NaN,$60 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
212,754442243360992989,"Tu casa en Neuquen, lo de Nano.",entire_home,NaN,Neuquén,-38.960444,-68.049372,"['No hay evaluaciones', '2 camas', '2 dormitor...",2023-07-22,2023-07-29,$53 USD,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
242,839758121679938488,Dpto en el centro de Cipolletti,entire_home,NaN,Cipolletti,-38.939177,-68.001222,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-09-04,2023-09-10,NaN,$79 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [950]:
# Carga de archivos de datos de Rio Negro
df_rio_negro = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Rio_Negro.csv')

df_rio_negro["provincia"] = "RIO NEGRO"

df_rio_negro.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
68,54094683,Cabaña en el bosque,entire_home,"4,9 (41)",San Carlos de Bariloche,-41.124250,-71.397970,"['1 cama', '1 dormitorio', '1 ,5 baños', 'Esta...",2023-08-17,2023-08-22,NaN,$72 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
52,46903374,"""Casita"" a estrenar, 21 m2 de luz",entire_home,"4,6 (40)",San Carlos de Bariloche,-41.090300,-71.452320,"['1 cama', 'Monoambiente', '1 cama doble', 'Es...",2023-07-16,2023-07-21,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
133,50586981,ACUARELA II en pleno centro,entire_home,"4,92 (39)",San Carlos de Bariloche,-41.136380,-71.323740,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-07,2023-07-12,NaN,$73 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
63,760003693579176617,Acogedora Cabaña B. Residencial con Vista al C...,entire_home,"5,0 (8)",Villa La Angostura,-40.747480,-71.660860,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-03,2023-07-10,$68 USD,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
167,14373763,Depto Malbec Céntrico c/parrilla y pileta,entire_home,"4,76 (135)",San Martin de los Andes,-40.154910,-71.344870,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-10,2023-07-17,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
159,31949732,"CABAÑA ESTILO KEY WEST, EN PATAGONIA",entire_home,"4,56 (46)",Dina Huapi,-41.066260,-71.159430,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-17,2023-07-22,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
45,21581081,"""Bathroom with LakeView Place and Retreat""",entire_home,"4,98 (127)",San Carlos de Bariloche,-41.067950,-71.450490,"['1 baño', '1 cama queen', '1 cama doble', 'má...",2023-08-10,2023-08-15,NaN,$160 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
28,53288809,Hermoso y acogedor depto con vista en el centro,entire_home,"4,62 (50)",San Carlos de Bariloche,-41.132720,-71.312380,"['1 baño', '1 cama doble', 'Wifi', 'Calificado...",2023-08-22,2023-08-27,NaN,$80 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
148,545022151966438713,Cómoda casa de huéspedes nueva con parrilla,entire_home,"5,0 (30)",San Carlos de Bariloche,-41.129550,-71.251060,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-10-01,2023-10-06,$69 USD,$81 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
184,755921214950226681,Casita vegetariana en la naturaleza,entire_home,"4,9 (31)",San Carlos de Bariloche,-41.047005,-71.515422,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-01,2023-07-06,$25 USD,$29 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [951]:
# Carga de archivos de datos de Tierra delFuego
df_tierra_del_fuego = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Tierra_del_Fuego.csv')

df_tierra_del_fuego["provincia"] = "TIERRA DEL FUEGO"

df_tierra_del_fuego.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
10,850856013943251401,Departamento Las Bandurrias,entire_home,"5,0 (7)",Ushuaia,-54.833140,-68.346220,"['4 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
97,48694593,RARE: casa del propietario actualizada y equip...,entire_home,"4,78 (45)",Ushuaia,-54.803000,-68.305160,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
12,53848130,Ocean & Mountain View,entire_home,"4,89 (9)",Ushuaia,-54.807163,-68.318520,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,$88 USD,$98 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
37,720428719363448238,Tu hogar en el fin del mundo! Ushuaia te espera!,entire_home,"5,0 (17)",Ushuaia,-54.821700,-68.354055,"['3 camas', 'Monoambiente', '1 baño', '2 camas...",2023-07-03,2023-07-08,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
17,604911298357239989,IBF SKY - Ushuaia Apartment,entire_home,"4,93 (27)",Ushuaia,-54.807125,-68.318497,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-01,2023-07-06,NaN,$95 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
242,18314403,"Depto: “El Jardín"" para 3 Personas!",entire_home,"4,86 (129)",Ushuaia,-54.813240,-68.341760,"['1 cama', '1 dormitorio', '1 ,5 baños', '1 ca...",2023-07-01,2023-07-06,NaN,$54 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
59,17142248,Planta alta VISTAMAR USHUAIA,entire_home,"4,68 (25)",Ushuaia,-54.839100,-68.369860,"['4 camas', '3 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,$203 USD,$228 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
169,547155610816481711,Villa de los Ñires,private_room,"5,0 (4)",Ushuaia,-54.837970,-68.368620,"['6 camas', '3 dormitorios', '2,5 baños', 'Wif...",2023-09-03,2023-09-08,NaN,$225 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
193,28273819,Amaneceres del Beagle,entire_home,"4,96 (47)",Ushuaia,-54.811960,-68.322300,"['3 camas', '1 dormitorio', '1 baño', '2 camas...",2023-09-12,2023-09-19,$79 USD,$119 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
128,865289942925242813,Abrazando el mar,entire_home,NaN,Río Grande,-53.757460,-67.726520,"['No hay evaluaciones', '3 camas', '3 dormitor...",2023-07-01,2023-07-06,NaN,$82 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [952]:
# Verificacion de columnas para concatenar
(df_chubut.columns == df_neuquen.columns) & (df_rio_negro.columns == df_tierra_del_fuego.columns) & (df_chubut.columns == df_tierra_del_fuego.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [953]:
# Concatenacion de los dataframes separados por provincias en un unico archivo
df_alquileres = pd.concat([df_chubut, df_neuquen, df_rio_negro, df_tierra_del_fuego])
df_alquileres.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
109,571693285038080905,Luminoso y vista al lago en el centro de Baril...,entire_home,"4,47 (49)",San Carlos de Bariloche,-41.134450,-71.316020,"['2 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-06,2023-07-11,NaN,$120 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
100,37496800,Increíble vista en el centro de Bariloche,entire_home,"4,79 (135)",San Carlos de Bariloche,-41.134130,-71.310850,"['2 baños', '1 cama doble', '2 camas de una pl...",2024-02-15,2024-02-20,NaN,$65 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
238,40349728,"La Pirucha, en Av Bustillo a 2 cuadras de la p...",entire_home,"4,72 (39)",San Carlos de Bariloche,-41.104670,-71.441320,"['3 camas', '2 dormitorios', '1 ,5 baños', '1 ...",2023-09-24,2023-09-29,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
57,29335293,"""Cipresales"" Cabaña Villa la Angostura Nqn",entire_home,"4,65 (149)",Villa La Angostura,-40.758260,-71.643790,"['3 camas', '2 dormitorios', '1 baño', '2 cama...",2023-08-09,2023-08-14,NaN,$87 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
89,37836757,Rincón del Sol,entire_home,"4,4 (5)",Puerto Madryn,-42.751650,-65.053520,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$18 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
89,579026916207068038,Monoambiente nivel ejecutivo zona centro Neuquén.,entire_home,NaN,Neuquén,-38.953239,-68.063423,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$36 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
217,33422894,Hermosa y cálida!,private_room,NaN,Neuquén,-38.944290,-68.047760,"['3 camas', '2 dormitorios', '1 baño compartid...",2023-07-01,2023-07-06,NaN,$24 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
55,788539781586101591,Bim 5,entire_home,NaN,Neuquén,-38.948340,-68.060020,"['2 camas', '1 dormitorio', '1 baño', 'Estacio...",2023-07-10,2023-07-15,NaN,$35 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
83,755135392594127489,Encantadoras vistas del canal de beagle - Amun...,entire_home,"5,0 (19)",Ushuaia,-54.810100,-68.319410,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
241,20234351,BARDO,shared_room,NaN,Neuquén,-38.943680,-68.053870,['No hay evaluaciones'],2023-07-01,2023-07-06,NaN,$10 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [954]:
# Reseteo del index, permitiendo duplicados porque aun no sabemos si son unicos los valores
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= True)
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= False)
df_alquileres.reset_index(drop=True,inplace=True)

*Verificacion de filas de los dataframes*

In [955]:
# Verificacion de la cantidad de filas de cada dataframe
print("La cantidad de filas del dataframe de Chubut es:"+str(df_chubut.shape[0]))
print("La cantidad de filas del dataframe de Neuquen es:"+str(df_neuquen.shape[0]))
print("La cantidad de filas del dataframe de Rio Negro es:"+str(df_rio_negro.shape[0]))
print("La cantidad de filas del dataframe de Tierra del fuego es:" + str(df_tierra_del_fuego.shape[0]))
print("La cantidad de filas del dataframe unificado de alquileres es:" + str(df_alquileres.shape[0]))


La cantidad de filas del dataframe de Chubut es:270
La cantidad de filas del dataframe de Neuquen es:270
La cantidad de filas del dataframe de Rio Negro es:270
La cantidad de filas del dataframe de Tierra del fuego es:270
La cantidad de filas del dataframe unificado de alquileres es:1080


*Verificacion de las columnas de los dataframes*

In [956]:
# Verificacion de la cantidad de columnas de cada dataframe
print("La cantidad de columnas del dataframe de Chubut es:" +
      str(df_chubut.shape[1]))
print("La cantidad de columnas del dataframe de Neuquen es:" +
      str(df_neuquen.shape[1]))
print("La cantidad de columnas del dataframe de Rio Negro es:" +
      str(df_rio_negro.shape[1]))
print("La cantidad de columnas del dataframe de Tierra del fuego es:" +
      str(df_tierra_del_fuego.shape[1]))
print("La cantidad de columnas del dataframe unificado de alquileres es:" +
      str(df_alquileres.shape[1]))

La cantidad de columnas del dataframe de Chubut es:18
La cantidad de columnas del dataframe de Neuquen es:18
La cantidad de columnas del dataframe de Rio Negro es:18
La cantidad de columnas del dataframe de Tierra del fuego es:18
La cantidad de columnas del dataframe unificado de alquileres es:18


*Verificacion general e inicial del dataframe unificado*

In [957]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1080 non-null   int64  
 1   name              1080 non-null   object 
 2   category          1080 non-null   object 
 3   rating            886 non-null    object 
 4   city              1076 non-null   object 
 5   latitude          1080 non-null   float64
 6   longitude         1080 non-null   float64
 7   characteristics   1080 non-null   object 
 8   check_in          1080 non-null   object 
 9   check_out         1080 non-null   object 
 10  price_discounted  201 non-null    object 
 11  price_original    1080 non-null   object 
 12  price_qualifier   1080 non-null   object 
 13  cant_resultados   1080 non-null   int64  
 14  fecha_scrapping   1080 non-null   object 
 15  fecha_cambio_bna  1080 non-null   object 
 16  cambio_bna        1080 non-null   float64


In [958]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.describe()

,id_alojamiento,latitude,longitude,cant_resultados,cambio_bna
count,1.080000e+03,1080.000000,1080.000000,1080.000000,1080.000000
mean,3.685341e+17,-44.426201,-68.723924,824.500000,239.919355
std,3.903051e+17,6.164758,2.159715,304.115744,0.000000
min,4.468310e+05,-54.844914,-71.688000,298.000000,239.919355
25%,3.529220e+07,-47.854503,-71.310280,824.500000,239.919355
50%,5.408344e+07,-42.003780,-68.305610,1000.000000,239.919355
75%,7.872495e+17,-39.609811,-68.049328,1000.000000,239.919355
max,9.198428e+17,-38.874370,-62.920960,1000.000000,239.919355


In [959]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.duplicated(keep="first").sum()

51

In [960]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.isna().sum()

id_alojamiento        0
name                  0
category              0
rating              194
city                  4
latitude              0
longitude             0
characteristics       0
check_in              0
check_out             0
price_discounted    879
price_original        0
price_qualifier       0
cant_resultados       0
fecha_scrapping       0
fecha_cambio_bna      0
cambio_bna            0
provincia             0
dtype: int64

*Conclusiones iniciales*
<br>
+ El dataframe unificado, coincide en cantidad de filas y columnas correspondientes a los dataframes individuales
+ Debemos de corregir algunos tipos de datos de columnas
+ Tenemos algunas columnas para reescalar
+ Tenemos filas duplicadas para revisar
+ Tenemos valores nulos para revisar, en principio para las columnas rating y price_discounted

**<h1>2. Análisis de duplicados</h1>**

*Analisis y tratamiento de duplicados global*

In [961]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres.duplicated(keep="first").sum()

51

In [962]:
#  Listo los valores duplicados
df_alquileres[df_alquileres.duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
54,40755147,La Amarilla Casa de Huéspedes (con cocina) Rut...,entire_home,"4,93 (27)",Epuyén,-42.227509,-71.362289,"['2 camas', '1 dormitorio', '1 ,5 baños', '1 c...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
72,586216477715336949,Encantador dpto. con vista al mar en Puerto Ma...,entire_home,"5,0 (11)",Puerto Madryn,-42.771633,-65.029068,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-07-06,2023-07-11,NaN,$131 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
90,37836757,Rincón del Sol,entire_home,"4,4 (5)",Puerto Madryn,-42.751650,-65.053520,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$18 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
113,736253141994173324,Refugio Ciprés con tina de agua caliente,entire_home,"5,0 (4)",Trevelin,-42.968260,-71.601320,"['1 cama', '1 dormitorio', '1 baño', 'Califica...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
127,21925703,Hermoso Duplex con Patio y Parrilla. (4 pax).,entire_home,"4,81 (74)",Puerto Madryn,-42.787700,-65.023850,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
128,644043911246503372,"Los Molles, casa familiar en entorno natural.",entire_home,NaN,Puerto Madryn,-42.735607,-65.100117,"['3 camas', '2 dormitorios', '1 baño']",2023-07-01,2023-07-07,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
129,883174783781736766,Espacio 2 - Rada Tilly,entire_home,Nuevo,Rada Tilly,-45.925810,-67.575650,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
138,53298317,Mar de Olivillos Planta alta,entire_home,"5,0 (12)",Puerto Piramides,-42.569200,-64.273610,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
162,31081731,"Domos Puelo, para dos, con jacuzzi",entire_home,"4,98 (40)",Lago Puelo,-42.041570,-71.592370,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-08,2023-07-15,NaN,$67 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
182,50704005,Hostel patagónico con vista 360 de la Cordillera,shared_room,"4,86 (7)",Trevelin,-43.071450,-71.454920,"['20 camas', '4 dormitorios', '3,5 baños compa...",2023-07-01,2023-07-06,NaN,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [963]:
# Elimino los duplicados, que coinciden en toda la fila
df_alquileres.drop_duplicates(keep="first", inplace=True)
df_alquileres.shape

(1029, 18)

*Analisis y tratamiento de duplicados por columna*

+ Columna 1: id_alojamiento

In [964]:
# Busco si hay duplicados
df_alquileres['id_alojamiento'].duplicated(keep='first').sum()

1

In [965]:
# Identifico filas con esos valores duplicados
df_alquileres[df_alquileres['id_alojamiento'].duplicated(keep='first')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [966]:
# Muestro las filas que corresponden a los valores duplicados, todas
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


-> Analisis de los valores de las filas duplicadas.

In [967]:
#Verificamos los valores 
df_alquileres.loc[647]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [968]:
# Verificamos los valores
df_alquileres.loc[648]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [969]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[647]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi']"

In [970]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[648]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi', 'Calificado con 5 de 5 en limpieza']"

*ACCION:*
<BR>
*Vemos que seria el mismo registro con excepcion de la serie characteristics, donde tiene agregada comentario de calificacion*
<br>
*Como consideramos que cada valor de id_alojamiento, deberia ser unico, eliminamos el que consideramos repetido y menos se adecua a los valores esperados por serie*

In [971]:
# Descartamos el registro que no tiene agregado el comentario de la calificacion
df_alquileres.drop([648], inplace=True)
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


+ Columna 2: name

In [972]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['name'].duplicated(keep="first").sum()

1

In [973]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres[df_alquileres['name'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.1333,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [974]:
df_alquileres[df_alquileres['name'].str.contains('Vista al Lago en el Bosque - Hab Compartida')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
712,797077259992479162,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13455,-71.36807,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-11,2023-07-18,NaN,$21 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13330,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


*ACCION:*
<br>
*En este caso vemos que los nombre coinciden pero, pueden corresponder a diferentes lugares, de hecho, estan sobre diferentes calles, segun sus coordenadas.*
<br>
*Por tal motivo los dejamos a ambos registros*

+ Columna 3: category

In [975]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['category'].duplicated(keep="first").sum()

1025

In [976]:
# Listo los duplicados segun category
df_alquileres[df_alquileres['category'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
148,50852832,Cálido depto. con patio en Esquel,entire_home,"5,0 (12)",Esquel,-42.915580,-71.332430,"['4 camas', '1 dormitorio', '1 baño', '2 camas...",2023-07-01,2023-07-06,NaN,$34 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
406,831164056448451302,Departamento con balcón,entire_home,NaN,Neuquén,-38.925274,-68.052651,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-17,2023-07-22,NaN,$37 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
485,878902407464873092,Monoambiente en suite,entire_home,Nuevo,Neuquén,-38.954110,-68.084270,"['2 camas', 'Monoambiente', '1 baño', 'Estacio...",2023-07-01,2023-07-06,$53 USD,$66 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
123,21925703,Hermoso Duplex con Patio y Parrilla. (4 pax).,entire_home,"4,81 (74)",Puerto Madryn,-42.787700,-65.023850,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
889,752426111954594687,Las carmelitas 1 Apartamento,entire_home,"4,96 (23)",Ushuaia,-54.815088,-68.329166,"['2 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-08-14,2023-08-19,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1052,18314403,"Depto: “El Jardín"" para 3 Personas!",entire_home,"4,86 (129)",Ushuaia,-54.813240,-68.341760,"['1 cama', '1 dormitorio', '1 ,5 baños', '1 ca...",2023-07-01,2023-07-06,NaN,$54 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
445,15753173,Moderno y acogedor departamento en zona centrica,entire_home,"4,91 (35)",Neuquén,-38.943120,-68.053160,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,$85 USD,$95 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
510,587675039019946875,Hermoso departamento en Neuquén,entire_home,NaN,Neuquén,-38.950830,-68.056560,"['4 camas', '3 dormitorios', '2 baños', 'Wifi']",2023-07-01,2023-07-06,NaN,$106 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
609,34865700,Acogedor apartamento en el centro,entire_home,"4,81 (119)",San Carlos de Bariloche,-41.135320,-71.306010,"['1 cama', '1 dormitorio', '1 cama doble', 'Wi...",2023-09-25,2023-10-02,$36 USD,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
21,25584751,"Apartamentos ""El Atardecer"" The Sunset """,entire_home,"4,66 (53)",Puerto Piramides,-42.568820,-64.278180,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$138 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que los valores de categoria se pueden repetir para los diferentes alquileres*
<br>
*por lo cual, no realizamos ninguna accion en base a esta columna*

+ Columna 4: rating

In [977]:
# Verifico los duplicado
df_alquileres['rating'].duplicated(keep="first").sum()

564

In [978]:
# Listamos algunas filas, repetidas por rating
df_alquileres[df_alquileres['rating'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
373,815951591179105029,Luminoso Departamento en Neuquén,entire_home,NaN,Neuquén,-38.948930,-68.043060,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$99 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
1022,807095276920265226,Nuevo Departamento Cómodo 2 Horizonte Ushuaia,entire_home,"5,0 (15)",Ushuaia,-54.808549,-68.338988,"['3 camas', '1 dormitorio', '1 baño', '3 camas...",2023-07-01,2023-07-06,NaN,$32 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
125,883174783781736766,Espacio 2 - Rada Tilly,entire_home,Nuevo,Rada Tilly,-45.925810,-67.575650,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1050,828090107927300389,GRUshuaia,entire_home,NaN,Ushuaia,-54.815088,-68.329166,"['No hay evaluaciones', '1 cama', 'Monoambient...",2023-09-13,2023-09-18,$51 USD,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
121,748631165790302932,Casa a metros del centro y terminal,entire_home,NaN,Puerto Madryn,-42.768445,-65.043064,"['2 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
480,870770974790974455,Depto 1 Dormitorio Neuquén,entire_home,Nuevo,Neuquén,-38.946465,-68.054645,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$68 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
787,889382807675939102,Acogedor departamento de 2 amb,entire_home,Nuevo,San Carlos de Bariloche,-41.122310,-71.369530,"['1 cama', '1 dormitorio', '1 baño', 'Estacion...",2023-07-05,2023-07-10,NaN,$62 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
706,734877502966665498,Casa Bosque,entire_home,NaN,San Martín de los Andes,-40.122360,-71.290410,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$110 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
744,649307420913365326,casa paqueta en las coloradas,entire_home,"5,0 (7)",Las Grutas,-40.828060,-65.113830,"['4 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$150 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
1056,710189161538779745,Apartamento en el centro de Ushuaia con vistas...,entire_home,"4,93 (14)",Ushuaia,-54.810360,-68.318870,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-29,2023-08-03,NaN,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [979]:
df_alquileres['rating'].unique()

array([nan, '5,0 (6)', '4,91 (70)', 'Nuevo', '4,96 (95)', '4,86 (7)',
       '4,82 (17)', '4,96 (75)', '4,91 (67)', '4,57 (14)', '4,6 (5)',
       '4,94 (18)', '4,95 (22)', '4,92 (64)', '4,94 (109)', '4,77 (44)',
       '4,89 (35)', '5,0 (7)', '4,66 (53)', '5,0 (34)', '4,98 (93)',
       '4,96 (51)', '4,82 (66)', '4,8 (51)', '4,83 (36)', '4,9 (20)',
       '5,0 (17)', '4,65 (49)', '4,7 (33)', '4,89 (9)', '4,97 (90)',
       '5,0 (10)', '4,79 (30)', '4,86 (74)', '4,86 (44)', '4,63 (128)',
       '5,0 (3)', '4,88 (16)', '4,81 (32)', '5,0 (9)', '4,93 (27)',
       '4,92 (13)', '4,5 (4)', '4,77 (13)', '4,67 (12)', '4,9 (176)',
       '4,95 (19)', '4,8 (15)', '5,0 (14)', '4,79 (14)', '5,0 (4)',
       '5,0 (19)', '4,67 (6)', '4,82 (137)', '5,0 (11)', '4,77 (102)',
       '4,47 (55)', '4,76 (134)', '4,74 (19)', '4,96 (27)', '4,91 (11)',
       '4,93 (15)', '4,62 (13)', '4,55 (33)', '4,4 (5)', '4,72 (25)',
       '4,58 (76)', '4,73 (11)', '4,83 (60)', '4,93 (28)', '4,7 (10)',
       '4,76 (85

*ACCION:*
<br>
*Entendemos que los valores de rating se pueden repetir para los diferentes alquileres*
<br>
*Por lo cual, dejamos los registros retpetidos segun esta columna*

+ Columna 5: city

In [980]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['city'].duplicated(keep="first").sum()

947

In [981]:
# Listamos repetidos, por city
df_alquileres[df_alquileres['city'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,NaN,Puerto Madryn,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
2,800749301724167838,Departamento Bella Vista,entire_home,"5,0 (6)",Puerto Madryn,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,"4,91 (70)",Puerto Madryn,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
4,916738743930751566,Front Beach,entire_home,Nuevo,Puerto Madryn,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
5,27877802,Luxury top 11th floor with the best 360 views,entire_home,"4,96 (95)",Puerto Madryn,-42.768330,-65.032100,"['4 camas', '2 dormitorios', '2,5 baños', '1 c...",2023-07-06,2023-07-11,NaN,$122 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,771725397497773450,Casa Fueguina en Centro,entire_home,"4,53 (17)",Ushuaia,-54.803310,-68.302580,"['3 camas', '2 dormitorios', '1 baño', 'Wifi',...",2023-08-14,2023-08-19,$60 USD,$76 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1076,26215568,Hospedaje UshuaiAndo,entire_home,"4,82 (50)",Ushuaia,-54.800790,-68.304230,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-23,2023-07-28,$45 USD,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1077,42486711,Yeska C Dpto. Supercéntrico. Casco Histórico,entire_home,"4,88 (8)",Ushuaia,-54.805430,-68.308710,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1078,51921207,KREN I,entire_home,"4,85 (26)",Ushuaia,-54.806170,-68.317800,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-10,2023-07-15,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna city, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 6: latitude

In [982]:
# Verificamos los duplicados
df_alquileres['latitude'].duplicated(keep="first").sum()

36

In [983]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['latitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
981,742698340271989675,"Excelente departamento, excelente ubicación.",entire_home,"4,94 (18)",Ushuaia,-54.810100,-68.318770,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-08-01,2023-08-06,NaN,$80 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
443,53105728,LOFT RESIDENCIAL ALTO COMAHUE DE COMPRAS!!,entire_home,NaN,Neuquén,-38.941450,-68.063080,"['No hay evaluaciones', '2 camas', 'Monoambien...",2023-07-01,2023-07-06,NaN,$20 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
657,52904852,Monoambiente 30m2 para 3. Céntrico. Calle Mitre!,entire_home,"4,89 (61)",San Carlos de Bariloche,-41.134030,-71.298840,"['3 camas', 'Monoambiente', '1 baño', '4 camas...",2023-10-07,2023-10-12,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
526,36144936,Dpto nuevo con aire en una hermosa zona.,entire_home,"4,89 (19)",Neuquén,-38.947090,-68.048020,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-01,2023-08-07,NaN,$48 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
109,905128221670422939,Habitación con Altillo para 4 Pa,private_room,Nuevo,Puerto Madryn,-42.755600,-65.037001,"['3 camas', '1 dormitorio', '1 baño compartido']",2023-07-01,2023-07-06,$14 USD,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1077,42486711,Yeska C Dpto. Supercéntrico. Casco Histórico,entire_home,"4,88 (8)",Ushuaia,-54.805430,-68.308710,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
427,651446721456756885,Depto hermoso con estacionamiento incluido.,entire_home,"4,83 (6)",Neuquén,-38.949400,-68.052530,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
709,17846750,Cabaña en Bariloche a minutos del Centro,entire_home,"4,89 (108)",San Carlos de Bariloche,-41.127640,-71.358010,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-08-09,2023-08-14,$75 USD,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
717,49546396,Departamento para 2. Excelente ubicación.,entire_home,"4,89 (53)",San Carlos de Bariloche,-41.134550,-71.299200,"['1 cama', 'Monoambiente', '1 baño', '1 cama q...",2023-07-07,2023-07-13,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
982,690413283545175725,Las Aguilas 3,entire_home,"4,67 (9)",Ushuaia,-54.814922,-68.341647,"['4 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-08,2023-07-14,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [984]:
# Vemos un ejemplo
df_alquileres[df_alquileres['latitude'] == -54.803600]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
934,31825003,Departamento con inmejorable ubicación en Ushuaia,entire_home,"4,9 (108)",Ushuaia,-54.8036,-68.30374,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1025,623178032761843625,"departamento céntrico, cálido y cómodo.",entire_home,"4,88 (16)",AIU,-54.8036,-68.32132,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$33 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna latitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 7: longitude

In [985]:
# Verificamos los duplicados
df_alquileres['longitude'].duplicated(keep="first").sum()

20

In [986]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['longitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1071,700712866756917764,D) Cuarto doble privado,private_room,"4,85 (13)",Ushuaia,-54.796494,-68.298038,"['2 camas', '1 dormitorio', '1,5 baños compart...",2023-07-01,2023-07-06,NaN,$26 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
15,767503846951993933,Brisa de Mar - Puerto Madryn,entire_home,"4,95 (22)",Puerto Madryn,-42.771634,-65.029071,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-08-07,2023-08-12,$34 USD,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
851,751455808581955668,Ushuaia54,entire_home,"4,77 (26)",Ushuaia,-54.808950,-68.319000,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,$59 USD,$68 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
86,51311028,"Amplio Monoambiente con estacionamiento,cerrado.",entire_home,"4,55 (33)",Esquel,-42.915770,-71.307150,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$20 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
109,905128221670422939,Habitación con Altillo para 4 Pa,private_room,Nuevo,Puerto Madryn,-42.755600,-65.037001,"['3 camas', '1 dormitorio', '1 baño compartido']",2023-07-01,2023-07-06,$14 USD,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1031,711709136406452484,F) Cama en cuarto mixto compartido,shared_room,"4,67 (9)",Ushuaia,-54.796494,-68.298038,"['1 cama', '1 dormitorio', '1,5 baños comparti...",2023-07-01,2023-07-06,NaN,$12 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1044,51921353,KREN II,entire_home,"4,77 (30)",Ushuaia,-54.807290,-68.316100,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
948,889733753812060778,Ushuaia Mágica - Ush AustraLis,entire_home,Nuevo,Ushuaia,-54.811890,-68.326950,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
243,909336244153347770,Bella Vista 2,entire_home,Nuevo,Esquel,-42.917491,-71.294685,"['4 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$52 USD,$65 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
531,833878451226342434,El lugar de Ana - Am. Comahue,entire_home,NaN,Neuquén,-38.953065,-68.065207,"['No hay evaluaciones', '3 camas', '1 dormitor...",2023-07-02,2023-07-08,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [987]:
# Vemos un ejemplo
df_alquileres[df_alquileres['longitude'] == -65.029470]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
70,20364091,Departamento Solmar,entire_home,"4,82 (137)",Puerto Madryn,-42.76963,-65.02947,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-10-15,2023-10-20,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
257,53295849,Preciosa unidad de alquiler de 3 dormitorios e...,entire_home,"4,84 (25)",Puerto Madryn,-42.77073,-65.02947,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-03,2023-07-08,NaN,$114 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que la columna longitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columan 8: characteristics

In [988]:
# Verificamos los duplicados
df_alquileres['characteristics'].duplicated(keep="first").sum()

243

In [989]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['characteristics'].duplicated(
    keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
908,829670957208558538,Departamento de Ana y José - 2 personas,entire_home,"5,0 (5)",Ushuaia,-54.803140,-68.316240,"['2 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-08-29,2023-09-03,NaN,$44 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
134,38028114,Fuego,entire_home,"4,86 (7)",Trelew,-43.262050,-65.302140,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-08-01,2023-08-07,NaN,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1043,34791477,AlSur | Vista al Canal Beagle y a las Montañas,entire_home,"4,96 (93)",Ushuaia,-54.803260,-68.325030,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-08-06,2023-08-11,$56 USD,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
217,594039328041406440,Postales del mar,entire_home,Nuevo,Puerto Madryn,-42.784660,-65.000810,"['3 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
450,826077606681970295,Don Bosco 1ero,entire_home,NaN,Neuquén,-38.966270,-68.066030,"['2 camas', '1 dormitorio', '1 baño', 'Estacio...",2023-07-01,2023-07-06,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
474,829574033573661357,"Villa María, don Bosco 1125.",entire_home,NaN,Neuquén,-38.967760,-68.064260,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$55 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
631,48806077,Casa Pajaro Carpintero,entire_home,"4,92 (37)",San Carlos de Bariloche,-41.070060,-71.488750,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,$90 USD,$100 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
1031,711709136406452484,F) Cama en cuarto mixto compartido,shared_room,"4,67 (9)",Ushuaia,-54.796494,-68.298038,"['1 cama', '1 dormitorio', '1,5 baños comparti...",2023-07-01,2023-07-06,NaN,$12 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
964,708125262725542828,Pequeño piso en exclusivo Ushuaia,entire_home,"4,88 (32)",Ushuaia,-54.809530,-68.316100,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-07-23,2023-07-30,$73 USD,$81 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
956,53699993,Moderno estudio con jardín,entire_home,"4,94 (54)",Ushuaia,-54.809040,-68.319680,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-10-23,2023-10-28,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna characteristics, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 9: check_in

In [990]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_in'].duplicated(keep="first").sum()

920

*ACCION:*
<br>
*Entendemos que la columna check_in, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 10: check_out

In [991]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_out'].duplicated(keep="first").sum()

916

*ACCION:*
<br>
*Entendemos que la columna check_out, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 11: price_discounted

In [992]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_discounted'].duplicated(keep="first").sum()

938

*ACCION:*
<br>
*Entendemos que la columna price_discounted, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 12: price_original

In [993]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_original'].duplicated(keep="first").sum()

868

*ACCION:*
<br>
*Entendemos que la columna price_original, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 13: price_qualifier

In [994]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_qualifier'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna price_qualifier, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 14: cant_resultados

In [995]:
# Verificamos los duplicados
df_alquileres['cant_resultados'].duplicated(keep="first").sum()

1026

*ACCION:*
<br>
*Entendemos que la columna cant_resultados, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

Columna 15: fecha_scrapping

In [996]:
# Verificamos los duplicados
df_alquileres['fecha_scrapping'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna fecha_scrapping, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 16: fecha_cambio_bna

In [997]:
# Verificamos los duplicados
df_alquileres['fecha_cambio_bna'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna fecha_cambio_bna, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 17: cambio_bna

In [998]:
# Verificamos los duplicados
df_alquileres['cambio_bna'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna cambio_bna, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 18: provincia

In [999]:
# Verificamos los duplicados
df_alquileres['provincia'].duplicated(keep="first").sum()

1024

*ACCION:*
<br>
*Entendemos que la columna provincia, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

*OBSERVACION GENERAL*
<br>
*En general entendemos que la mayoria de las columnas podrian tener valores repetidos evaluandolas individualmente.*
<br>
*Pero si consideramos que la variable id_alojamiento identifica de manera biunivoca un alojamiento.*
<br>
*Podemos decir que no es lo mismo, un duplicado en la columna id_alojamiento(que no deberia tener valores repetidos), que un duplicado en la columna price_original,*
<br>
*(que podria perfectamente tener valores repetidos en la seria).*


**<h1>3. Análisis de datos faltantes y limpieza</h1>**

+ Mostrar el porcentaje de datos faltantes de cada columna

In [1000]:
# Verificacion inicial del dataset
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   category          1028 non-null   object 
 3   rating            842 non-null    object 
 4   city              1024 non-null   object 
 5   latitude          1028 non-null   float64
 6   longitude         1028 non-null   float64
 7   characteristics   1028 non-null   object 
 8   check_in          1028 non-null   object 
 9   check_out         1028 non-null   object 
 10  price_discounted  191 non-null    object 
 11  price_original    1028 non-null   object 
 12  price_qualifier   1028 non-null   object 
 13  cant_resultados   1028 non-null   int64  
 14  fecha_scrapping   1028 non-null   object 
 15  fecha_cambio_bna  1028 non-null   object 
 16  cambio_bna        1028 non-null   float64
 17  

*En primera instancia observamos que solo nos quedan dos columnas con datos faltantes: rating y price_discounted*
<br>
*Para rating, tenenmos un 81.90 % de faltantes (842 de 1028).*
<br>
*Para rating, tenenmos un 18.58 % de faltantes (191 de 1028).*


+ ¿Cuáles son las 3 columnas con más datos faltantes?

*En nuestra observacion inicial, en principio tenemos dos columnas con datos faltantes solamente: rating y price_discounted*

**LIMPIEZA DE DE DATOS POR COLUMNAS**

In [1001]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   category          1028 non-null   object 
 3   rating            842 non-null    object 
 4   city              1024 non-null   object 
 5   latitude          1028 non-null   float64
 6   longitude         1028 non-null   float64
 7   characteristics   1028 non-null   object 
 8   check_in          1028 non-null   object 
 9   check_out         1028 non-null   object 
 10  price_discounted  191 non-null    object 
 11  price_original    1028 non-null   object 
 12  price_qualifier   1028 non-null   object 
 13  cant_resultados   1028 non-null   int64  
 14  fecha_scrapping   1028 non-null   object 
 15  fecha_cambio_bna  1028 non-null   object 
 16  cambio_bna        1028 non-null   float64
 17  

+ Columna 0: id_alojamiento

In [1002]:
df_alquileres['id_alojamiento'].nunique()

1028

In [1003]:
df_alquileres['id_alojamiento'].isna().sum()

0

+ Columna 1: name

In [1004]:
sorted(df_alquileres['name'].unique())

['"Alegria" vistas de ensueño en un lugar para enamorarse',
 '"Bathroom with LakeView Place and Retreat"',
 '"Casita" a estrenar, 21 m2 de luz',
 '"Cerros Nevados" departamento tipo loft.',
 '"Cipresales"     Cabaña  Villa la Angostura  Nqn',
 '"Doña Cota" - Amueblados Comahue',
 '"El Rancho" Sentirte en casa',
 '"La Encantada" en Villa los Coihues',
 '"La Posadita" Casa de Montaña, Miralejos',
 '"La casa de Cielo "',
 '"Los Amancays "\nCabaña de montaña',
 '"Tiny House de Ensueño"',
 '"Ty Bychan" \nLa Casita. Entre raíces galesas.',
 '0 - Pura Tranquilidad en el Corazón de Neuquén',
 '1- Pura Tranquilidad en el Corazón de Neuquén',
 '11-E Espectacular departamento Alto Neuquen',
 '2 - Pura tranquilidad en el corazón de Neuquén',
 '2 Amigos de viaje en un lugar comodo',
 '2 Habitaciones. 4 Personas en Casa con Pileta',
 '2 Personas. Lugar súper cómodo',
 '2 ambientes en el centro',
 '2x1 Central Family',
 '3 dormitorios. cama matrimonial + 3 de una plaza',
 '3- Pura tranquilidad en el 

+ Columna 2: category

In [1005]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['category'].unique()

array(['entire_home', 'private_room', 'shared_room'], dtype=object)

In [1006]:
# Verificamos que no tienen  na
df_alquileres['category'].isna().sum()

0

In [1007]:
# Cambiamos el tipo de la serie
df_alquileres['category'] = df_alquileres['category'].astype('category')

+ Columna 3: rating

In [1008]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['rating'].unique()

array([nan, '5,0 (6)', '4,91 (70)', 'Nuevo', '4,96 (95)', '4,86 (7)',
       '4,82 (17)', '4,96 (75)', '4,91 (67)', '4,57 (14)', '4,6 (5)',
       '4,94 (18)', '4,95 (22)', '4,92 (64)', '4,94 (109)', '4,77 (44)',
       '4,89 (35)', '5,0 (7)', '4,66 (53)', '5,0 (34)', '4,98 (93)',
       '4,96 (51)', '4,82 (66)', '4,8 (51)', '4,83 (36)', '4,9 (20)',
       '5,0 (17)', '4,65 (49)', '4,7 (33)', '4,89 (9)', '4,97 (90)',
       '5,0 (10)', '4,79 (30)', '4,86 (74)', '4,86 (44)', '4,63 (128)',
       '5,0 (3)', '4,88 (16)', '4,81 (32)', '5,0 (9)', '4,93 (27)',
       '4,92 (13)', '4,5 (4)', '4,77 (13)', '4,67 (12)', '4,9 (176)',
       '4,95 (19)', '4,8 (15)', '5,0 (14)', '4,79 (14)', '5,0 (4)',
       '5,0 (19)', '4,67 (6)', '4,82 (137)', '5,0 (11)', '4,77 (102)',
       '4,47 (55)', '4,76 (134)', '4,74 (19)', '4,96 (27)', '4,91 (11)',
       '4,93 (15)', '4,62 (13)', '4,55 (33)', '4,4 (5)', '4,72 (25)',
       '4,58 (76)', '4,73 (11)', '4,83 (60)', '4,93 (28)', '4,7 (10)',
       '4,76 (85

In [1009]:
# Observamos que existen valores na y uno que seria "Nuevo"
# Reemplazamos el valor Nuevo por na
mask = df_alquileres['rating'].str.lower().str.contains('nuevo', na = False)
df_alquileres.loc[mask, 'rating'] = np.NaN

In [1010]:
# Tratamiento de la columna rating
# Separamos los valores correspondientes para limpiarlos
df_rating= df_alquileres['rating'].str.split(pat="[ ]",expand=True).add_prefix('Columna')
# Renombramos las columnas
df_rating.rename(columns={"Columna0": "rating", "Columna1": "votaciones"}, inplace=True)

In [1011]:
# Limpiamos los valores para la columna rating
df_rating['rating'] = df_rating['rating'].replace(',','.')

In [1012]:
# Reemplazamos la columna por punto para la columna rating y lo cambiamos de tipo
df_rating['rating'] = df_rating['rating'].str.replace(',', '.')
df_rating['rating'] = df_rating['rating'].astype('float64')

In [1013]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rating      726 non-null    float64
 1   votaciones  726 non-null    object 
dtypes: float64(1), object(1)
memory usage: 24.1+ KB


In [1014]:
# Limpiamos la columna de votaciones para cambiarle el tipo
df_rating['votaciones'] = df_rating['votaciones'].str.replace('(', '').str.replace(')', '')
df_rating['votaciones'] = np.floor(pd.to_numeric(df_rating['votaciones'], errors='coerce')).astype('Int64')


In [1015]:
#  Rellenamos los valores para las columnas rating y votaciones
# Rellenamos con un valor promedio, pero establecemos que no tiene votaciones
media_rating = df_rating['rating'].mean()
media_votaciones = np.floor(df_rating['votaciones'].mean())
df_rating.fillna({'rating':media_rating,'votaciones':media_votaciones},inplace= True)


In [1016]:
# Enriquecemos el dataframe de alquileres con las columnas de rating y votaciones
df_alquileres= df_alquileres.merge(df_rating,how='left',left_index=True,right_index=True,suffixes=('', '_y'))
df_alquileres[['rating_y','votaciones']].head()


,rating_y,votaciones
0,4.846171,44
1,4.846171,44
2,5.000000,6
3,4.910000,70
4,4.846171,44


In [1017]:
# Actualizamos las columnas del dataframe rating y votaciones
df_alquileres['rating']= df_alquileres['rating_y']
df_alquileres = df_alquileres.drop('rating_y',axis=1)
df_alquileres.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_alojamiento    1028 non-null   int64   
 1   name              1028 non-null   object  
 2   category          1028 non-null   category
 3   rating            1028 non-null   float64 
 4   city              1024 non-null   object  
 5   latitude          1028 non-null   float64 
 6   longitude         1028 non-null   float64 
 7   characteristics   1028 non-null   object  
 8   check_in          1028 non-null   object  
 9   check_out         1028 non-null   object  
 10  price_discounted  191 non-null    object  
 11  price_original    1028 non-null   object  
 12  price_qualifier   1028 non-null   object  
 13  cant_resultados   1028 non-null   int64   
 14  fecha_scrapping   1028 non-null   object  
 15  fecha_cambio_bna  1028 non-null   object  
 16  cambio_bna        1028 non-nu

+ Agregamos la columna: best_reviews

In [1018]:
df_alquileres['best_reviews'] = np.where(df_alquileres['rating']==5.0,True,False)
df_alquileres.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones,best_reviews
62,559507142939465264,Duplex Playa Union- A tan solo 150mts del mar,entire_home,4.800000,Playa Unión,-43.326390,-65.051090,"['3 camas', '2 dormitorios', '2 baños', 'Calif...",2023-07-01,2023-07-06,NaN,$32 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,15,False
522,764436631477924880,Casa en Barrio Los Arcos,entire_home,4.846171,Plottier,-38.944240,-68.192180,"['No hay evaluaciones', '2 camas', '1 dormitor...",2023-07-01,2023-07-06,NaN,$40 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN,44,False
223,670311918240376780,Mar y Ballenas. Gales,entire_home,4.880000,Puerto Madryn,-42.770343,-65.033005,"['3 camas', '2 dormitorios', '2 baños', 'Calif...",2023-07-01,2023-07-06,$80 USD,$140 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,8,False
403,760932256894779814,Centro - BIM 5°,entire_home,4.846171,Neuquén,-38.948731,-68.060860,"['2 camas', '1 dormitorio', '1 baño', 'Wifi']",2023-07-10,2023-07-15,NaN,$45 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN,44,False
263,31021085,Un lugar en contacto con la naturaleza,entire_home,4.500000,Lago Puelo,-42.080110,-71.610830,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,10,False
962,730606158433979341,Terra San Martín,entire_home,4.970000,Ushuaia,-54.809440,-68.314360,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-08-01,2023-08-07,NaN,$81 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO,30,False
1036,871137571990477669,las aljabas,entire_home,4.846171,Ushuaia,-54.797290,-68.310018,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-23,2023-07-29,$36 USD,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO,44,False
916,605864319569019751,Las Rosas Dpto,entire_home,4.810000,BDP,-54.807210,-68.308450,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO,42,False
456,39766652,Pausa a 5 minutos del centro,entire_home,4.846171,Neuquén,-38.921920,-68.052940,"['3 camas', '2 dormitorios', '1 ,5 baños', '1 ...",2023-07-01,2023-07-06,NaN,$55 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN,44,False
160,735435828374047870,A una cuadra del mar con estacionamiento gratuito,private_room,3.800000,Puerto Piramides,-42.569840,-64.286020,"['1 cama', '1 dormitorio', '1 baño privado']",2023-07-01,2023-07-06,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,5,False


In [1019]:
# Porcentaje de moejores reviews
mejores = df_alquileres[df_alquileres["best_reviews"]]['best_reviews'].count()
no_mejores = df_alquileres[~df_alquileres["best_reviews"]]['best_reviews'].count()
porc = mejores /(mejores+no_mejores)*100
print(f"La cantidad de mejores reseñas es de :{mejores} y corresponde a un {porc} %.")

La cantidad de mejores reseñas es de :161 y corresponde a un 15.66147859922179 %.


+ Columna 4: city

In [1020]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['city'].unique()

array(['Puerto Madryn', 'Comodoro Rivadavia', 'Trelew', 'Esquel',
       'Puerto Piramides', 'Puerto Pirámides', 'Trevelin', 'Cholila',
       'Epuyén', 'Playa Unión', 'Lago Puelo', 'Playa Union', 'Rada Tilly',
       'Rawson', 'Biedma', 'Esquel,', 'Gaiman', 'El Hoyo', 'Cushamen',
       'Playa Magagna', 'Camarones', 'Biedma Department',
       'Gobernador Costa', 'Corcovado, Chubut', 'Neuquén', 'Neuquen',
       'BVH', 'Cipolletti', 'MDJ', 'Plottier', 'Confluencia',
       'Centenario', nan, 'Q8302ATE', 'JKB', 'Las Perlas', 'KCN',
       'Q8300MDJ', 'Confluencia Department', 'San Martín de Los Andes',
       'BGJ', 'Villa Traful', 'Villa La Angostura',
       'San Carlos de Bariloche', 'Bariloche', 'San Martin de los Andes',
       'Dina Huapi', 'Lácar', 'San Martín de los Andes', 'La Villa',
       'San Carlos de Bariloche-  Península de San Pedro',
       'Villa la Angostura ', 'Los Lagos', 'Puerto Manzano', 'Bariloche ',
       'San Carlos de Bariloche, Río Negro, AR', 'Viedma',
  

In [1021]:
# Como vemos que varias ciudades son la misma pero con alguna diferencia tipografica, tratamos de normalizar los valores
df_alquileres['city'] = df_alquileres['city'].str.strip().str.upper()
df_alquileres.head()

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones,best_reviews
0,855050336912355203,North Beach,entire_home,4.846171,PUERTO MADRYN,-42.756301,-65.037517,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-12,NaN,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,4.846171,PUERTO MADRYN,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
2,800749301724167838,Departamento Bella Vista,entire_home,5.000000,PUERTO MADRYN,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,6,True
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,4.910000,PUERTO MADRYN,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,70,False
4,916738743930751566,Front Beach,entire_home,4.846171,PUERTO MADRYN,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False


In [1022]:
# Quitamos los tildes
df_alquileres['city'] = df_alquileres['city'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_alquileres.head()

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones,best_reviews
0,855050336912355203,North Beach,entire_home,4.846171,PUERTO MADRYN,-42.756301,-65.037517,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-12,NaN,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,4.846171,PUERTO MADRYN,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
2,800749301724167838,Departamento Bella Vista,entire_home,5.000000,PUERTO MADRYN,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,6,True
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,4.910000,PUERTO MADRYN,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,70,False
4,916738743930751566,Front Beach,entire_home,4.846171,PUERTO MADRYN,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False


In [1023]:
# Recopilamos los datos de ciudades que tienen lugeres turisicos, segun la provincia, para tratar de agruparlas
aux_ciudades = pd.read_csv('Provincias_Ciudades.csv',sep= ';',encoding='latin1')
aux_ciudades['ciudad'] = aux_ciudades['ciudad'].str.upper().str.strip()
aux_ciudades['ciudad'] = aux_ciudades['ciudad'].str.normalize(
    'NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
aux_ciudades.head()

,ciudad,provincia
0,ALTO RIO SENGUER,CHUBUT
1,BAHIA BUSTAMANTE,CHUBUT
2,CAMARONES,CHUBUT
3,CHOLILA,CHUBUT
4,COMODORO RIVADAVIA,CHUBUT


In [1024]:
# Creamos una columna city2, que va a a guardar los valores resultantes para city
df_alquileres['city2'] = df_alquileres['city']


# Parametros para los cuales buscar la mejor aproximacion por fuzzy
provincias = ['TIERRA DEL FUEGO', 'NEUQUEN', 'RIO NEGRO', 'CHUBUT']
# Valres de match que aceptamos
aprobaciones = [80, 80, 80, 80]

# rr=0;
# iter=0
# Iteramos por indice, para provincias y aprobaciones
for indice in range((len(provincias))):
    # Iteramos  por cada ciudad segun la provincia
    for _, row in df_alquileres.loc[df_alquileres['provincia']==provincias[indice]].iterrows():
        ciudad_probar = str(row['city'])
        #aloj=-1
        #iter+=1
        max_coincidencia=0
        encontro = False
        # Recorremos la lista de ciudades de la provincia recabados para verificar
        for valor_correcto in aux_ciudades.loc[aux_ciudades['provincia'] == provincias[indice]]['ciudad']:            
            # Obtenemos el ratio de coincidencia
            coincidencia = fuzz.ratio(ciudad_probar, valor_correcto)
            #print(f"Iteraccion: {iter},id_Alojamiento:{row['id_alojamiento']} index: {_} ==== Comparo {ciudad_probar} de {str(row['provincia'])} , con {valor_correcto} de {provincias[indice]}. Resultdo {coincidencia}")
            # Si el puntaje de la coincidencia es mayor que la vigente
            if coincidencia > max_coincidencia:
                # Guardar el valor correcto
                correccion = valor_correcto
                # Reemplazar el puntaje
                max_coincidencia = coincidencia                
                if (coincidencia > max_coincidencia) & (max_coincidencia >= aprobaciones[indice]):
                    # Guardar el valor correcto
                    correccion = valor_correcto
                    # Reemplazar el puntaje
                    max_coincidencia = coincidencia
                    # Reemplazo por el mejor valor aproximado, en la columna auxiliar
                    df_alquileres['city2']=correccion
                    # Aviso que encontre algo
                    encontro = True
        # Si no encontre una coincidencia suficiente, cambio de macheo
        if( not encontro):
            # Recorremos la lista de departamentos
            max_coincidencia_parcial = 0            
            for valor_correcto_parcial in aux_ciudades.loc[aux_ciudades['provincia'] == provincias[indice]]['ciudad']:
                # Obtenemos el ratio de coincidencia
                coincidencia_parcial = fuzz.partial_ratio(ciudad_probar, valor_correcto_parcial)
                # print(f"Iteraccion: {iter},id_Alojamiento:{row['id_alojamiento']} index: {_} ==== Comparo {ciudad_probar} de {str(row['provincia'])} , con {valor_correcto_parcial} de {provincias[indice]}. Resultdo {coincidencia_parcial}")
                # Si el puntaje de la coincidencia es mayor que la vigente
                if coincidencia_parcial > max_coincidencia_parcial:
                    # Guardar el valor correcto
                    correccion_parcial = valor_correcto_parcial
                    # Reemplazar el puntaje
                    max_coincidencia_parcial = coincidencia_parcial
                    if (coincidencia_parcial > max_coincidencia_parcial) & (max_coincidencia_parcial >= aprobaciones[indice]):
                        # Guardar el valor correcto
                        correccion_parcial = valor_correcto_parcial
                        # Reemplazar el puntaje
                        max_coincidencia_parial = coincidencia_parcial                        
                        # Asigno el mejor valor posible encontrado
                        df_alquileres['city2']=correccion_parcial
                        encontro = True
                        
                        
# En caso de que no se cumpla con las condiciones del macheo, mantiene el valor para ciudad inicial
            

In [1025]:
# Tenemos cuatro alojamientos que no tienen ciudad ingresada
df_alquileres['city'].isna().sum()

4

In [1026]:
# Consideramos que la ciudad es un dato importante, y por la cantidad la accion a tomar es elimnar esos 4 registros. Pero como en este analisis no require el uso de la columna city. lo dejamos
# df_alquileres.dropna(subset=['city'],inplace= True)

In [1027]:
df_alquileres['city']=df_alquileres['city2']
df_alquileres = df_alquileres.drop('city2', axis=1)

In [1028]:
df_alquileres['city'].unique()

array(['PUERTO MADRYN', 'COMODORO RIVADAVIA', 'TRELEW', 'ESQUEL',
       'PUERTO PIRAMIDES', 'TREVELIN', 'CHOLILA', 'EPUYEN', 'PLAYA UNION',
       'LAGO PUELO', 'RADA TILLY', 'RAWSON', 'BIEDMA', 'ESQUEL,',
       'GAIMAN', 'EL HOYO', 'CUSHAMEN', 'PLAYA MAGAGNA', 'CAMARONES',
       'BIEDMA DEPARTMENT', 'GOBERNADOR COSTA', 'CORCOVADO, CHUBUT',
       'NEUQUEN', 'BVH', 'CIPOLLETTI', 'MDJ', 'PLOTTIER', 'CONFLUENCIA',
       'CENTENARIO', nan, 'Q8302ATE', 'JKB', 'LAS PERLAS', 'KCN',
       'Q8300MDJ', 'CONFLUENCIA DEPARTMENT', 'SAN MARTIN DE LOS ANDES',
       'BGJ', 'VILLA TRAFUL', 'VILLA LA ANGOSTURA',
       'SAN CARLOS DE BARILOCHE', 'BARILOCHE', 'DINA HUAPI', 'LACAR',
       'LA VILLA', 'SAN CARLOS DE BARILOCHE-  PENINSULA DE SAN PEDRO',
       'LOS LAGOS', 'PUERTO MANZANO',
       'SAN CARLOS DE BARILOCHE, RIO NEGRO, AR', 'VIEDMA',
       'LACAR DEPARTMENT', 'VILLA LA ANGOSTURA, NEUQUEN ARGENTINA',
       'LAS GRUTAS', 'EL BOLSON', 'JUNIN DE LOS ANDES',
       'LOS LAGOS DEPARTMENT'

In [1029]:
# Cambiamos el tipo de datos de city
df_alquileres['city'] = df_alquileres['city'].astype('category')

+ Columna 5: latitude

In [1030]:
df_alquileres['latitude'].info()

<class 'pandas.core.series.Series'>
Index: 1028 entries, 0 to 1079
Series name: latitude
Non-Null Count  Dtype  
--------------  -----  
1028 non-null   float64
dtypes: float64(1)
memory usage: 16.1 KB


In [1031]:
df_alquileres['latitude'].unique()

array([-42.7563015 , -42.75577473, -42.76800685, -42.77759   ,
       -42.77054   , -42.76833   , -42.78191   , -42.76473   ,
       -42.7716337 , -42.7538    , -42.76864   , -45.81441   ,
       -42.76506   , -42.76887   , -43.25534   , -42.91027   ,
       -42.76985   , -42.767826  , -42.77496   , -45.86703   ,
       -42.56882   , -43.27444   , -43.25614   , -42.77106   ,
       -45.86496   , -42.91138   , -42.77884   , -42.9148784 ,
       -42.9132481 , -42.91526   , -42.9074621 , -42.7687445 ,
       -42.56749   , -43.09089   , -43.25905   , -42.76854884,
       -45.86038   , -42.91053   , -42.772861  , -42.65054   ,
       -42.76724   , -43.2591    , -42.77329   , -42.91809   ,
       -42.92146   , -42.76916   , -42.752541  , -42.5117    ,
       -42.77835745, -43.25566171, -42.7756997 , -42.90797   ,
       -42.22750854, -42.77086   , -42.91956   , -42.99143   ,
       -42.56918   , -42.77094   , -42.57219   , -42.56978   ,
       -43.32639   , -43.25009918, -42.77208   , -42.76

In [1032]:
# No tenemos valores nulos
df_alquileres['latitude'].isna().unique()

array([False])

+ Columna 6: longitude

In [1033]:
df_alquileres['longitude'].info()

<class 'pandas.core.series.Series'>
Index: 1028 entries, 0 to 1079
Series name: longitude
Non-Null Count  Dtype  
--------------  -----  
1028 non-null   float64
dtypes: float64(1)
memory usage: 16.1 KB


In [1034]:
# No tenemos valores nulos
# df_alquileres['longitude'].isna().unique()
df_alquileres['longitude'].unique()

array([-65.0375173 , -65.03726515, -65.03310833, ..., -68.30871   ,
       -68.3178    , -68.27647   ])

In [1035]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_alojamiento    1028 non-null   int64   
 1   name              1028 non-null   object  
 2   category          1028 non-null   category
 3   rating            1028 non-null   float64 
 4   city              1024 non-null   category
 5   latitude          1028 non-null   float64 
 6   longitude         1028 non-null   float64 
 7   characteristics   1028 non-null   object  
 8   check_in          1028 non-null   object  
 9   check_out         1028 non-null   object  
 10  price_discounted  191 non-null    object  
 11  price_original    1028 non-null   object  
 12  price_qualifier   1028 non-null   object  
 13  cant_resultados   1028 non-null   int64   
 14  fecha_scrapping   1028 non-null   object  
 15  fecha_cambio_bna  1028 non-null   object  
 16  cambio_bna        1028 non-nu

+ Columna 7: characteristics

In [1036]:
df_alquileres['characteristics'].unique()

array(["['2 camas', '1 dormitorio', '1 baño']",
       "['2 camas', '1 dormitorio', '1 baño', '1 cama doble', '1 sofá cama']",
       "['3 camas', '1 dormitorio', '1 baño', 'Calificado con 5 de 5 en limpieza']",
       "['1 cama', '1 dormitorio', '1 baño', '1 cama queen', 'Calificado con 4,8 de 5 en limpieza']",
       "['2 camas', 'Monoambiente', '1 baño']",
       "['4 camas', '2 dormitorios', '2,5 baños', '1 cama king', '3 camas de una plaza', 'Calificado con 4,9 de 5 en limpieza']",
       "['3 camas', '2 dormitorios', '1 baño', 'Calificado con 4,6 de 5 en limpieza']",
       "['2 camas', 'Monoambiente', '1 baño', 'Calificado con 4,8 de 5 en limpieza']",
       "['3 camas', '2 dormitorios', '3 baños']",
       "['3 camas', '1 dormitorio', '1 baño', '2 camas queen', '1 sofá cama', 'Calificado con 5 de 5 en limpieza']",
       "['2 camas', '1 dormitorio', '1 baño', '1 cama doble', '1 sofá cama', 'Calificado con 4,9 de 5 en limpieza']",
       "['2 camas', '1 dormitorio', '1 ,5 baños'

In [1037]:
df_caracteristicas = df_alquileres[['id_alojamiento','characteristics']]
df_caracteristicas.sample(10)

,id_alojamiento,characteristics
397,22523381,"['4 camas', '1 dormitorio', '1 baño', '4 camas..."
637,48039403,"['1 cama', '1 dormitorio', '1 baño', '1 cama d..."
936,40149949,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '..."
134,38028114,"['2 camas', '1 dormitorio', '1 baño', '1 cama ..."
1035,40282621,"['3 camas', '1 dormitorio', '1 baño compartido..."
107,39340707,"['3 camas', '1 dormitorio', '1 baño', '1 cama ..."
145,41171673,"['8 camas', '3 dormitorios', '2 baños', '2 cam..."
734,47975732,"['5 camas', '3 dormitorios', '2 baños', '1 cam..."
1040,40002691,"['2 camas', 'Monoambiente', '1 baño', '1 cama ..."
916,605864319569019751,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ..."


In [1038]:
df_temp = df_caracteristicas['characteristics'].str.split(
    pat=',', expand=True).add_prefix('Columna')
df_caracteristicas = df_caracteristicas.merge(
    df_temp, how='left', left_index=True, right_index=True, suffixes=('', '_y'))

In [1039]:
import re

df_caracteristicas['cama']=None
df_caracteristicas['dormitorio']=None
df_caracteristicas['baño']=None

# df_caracteristicas['cama']=df_caracteristicas['cama'].astype('Int64')
# df_caracteristicas['dormitorio']=df_caracteristicas['dormitorio'].astype('Int64')
# df_caracteristicas['baño']=df_caracteristicas['baño'].astype('Int64')


# df_caracteristicas.head()
busqueda = ['cama', 'dormitorio', 'baño']
for bus in busqueda:
    for indice, fila in df_caracteristicas.iterrows():
        ii = 2
        while (pd.isnull(df_caracteristicas.loc[indice,bus]) and (ii <= 14)):
            cc = 'Columna'+str(ii-2)
            idd = fila['id_alojamiento']
            val = fila[cc]
            if(not pd.isnull(val)):
                if bool(re.search(r'\d '+bus,val)):
                    dato = re.search(r'\d',val)                    
                    print(fila['id_alojamiento'], val,dato.group(0))
                    df_caracteristicas.loc[indice,bus]=dato.group(0)
            ii+=1
            

855050336912355203 ['2 camas' 2
51484364 ['2 camas' 2
800749301724167838 ['3 camas' 3
50769917 ['1 cama' 1
916738743930751566 ['2 camas' 2
27877802 ['4 camas' 4
710001905979665306 ['3 camas' 3
733872239559572922 ['2 camas' 2
771163634190966787 ['3 camas' 3
29901714 ['3 camas' 3
53474704 ['2 camas' 2
648024232660783653 ['2 camas' 2
892142653549224219 ['2 camas' 2
838330910169982904 ['2 camas' 2
615210210554179091 ['3 camas' 3
767503846951993933 ['3 camas' 3
601433498805560904 ['3 camas' 3
37527056 ['4 camas' 4
53472591 ['3 camas' 3
53891791 ['3 camas' 3
49030081 ['1 cama' 1
25584751 ['4 camas' 4
38289617 ['2 camas' 2
653750937066514352 ['3 camas' 3
40003837 ['3 camas' 3
24102728 ['2 camas' 2
32919267 ['2 camas' 2
7931620 ['4 camas' 4
890430561594058377 ['2 camas' 2
882699956948235601 ['2 camas' 2
53204199 ['2 camas' 2
702174271336760773 ['4 camas' 4
882608055979409957 ['6 camas' 6
37128706 ['1 cama' 1
53493881 ['2 camas' 2
23445902 ['4 camas' 4
897657557762504119 ['2 camas' 2
6580105389

In [1047]:
df_caracteristicas.sample(20)


,id_alojamiento,characteristics,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,Columna10,Columna11,Columna12,cama,dormitorio,baño
920,27197171,"['5 camas', '3 dormitorios', '2 baños', '1 cam...",['5 camas','3 dormitorios','2 baños','1 cama doble','1 cama de una plaza','más','Wifi','Lavarropas','Cocina','Calefacción','Calificado con 4,8 de 5 en limpieza'],None,5,3,2
478,35213605,"['No hay evaluaciones', '1 cama', '1 dormitori...",['No hay evaluaciones','1 cama','1 dormitorio','1 baño','1 cama doble','Wifi','No hay evaluaciones'],None,None,None,None,None,None,1,1,1
1069,738224105784766502,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",['2 camas','1 dormitorio','1 baño','1 cama doble','1 cama de una plaza','Wifi','Lavarropas','Cocina','Calefacción','Calificado con 4,8 de 5 en limpieza'],None,None,2,1,1
275,42407010,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",['2 camas','1 dormitorio','1 baño','1 cama doble','1 cama de una plaza','Estacionamiento gratis','Wifi','Calificado con 4,8 de 5 en limpieza'],None,None,None,None,2,1,1
544,35064808,"['3 camas', '2 dormitorios', '1 baño', 'Estaci...",['3 camas','2 dormitorios','1 baño','Estacionamiento gratis','Wifi','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,3,2,1
1025,623178032761843625,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",['3 camas','1 dormitorio','1 baño','Wifi','Cocina','Calefacción','Calificado con 4,9 de 5 en limpieza'],None,None,None,None,None,3,1,1
589,49107072,"['1 cama', '1 dormitorio', '1 baño', '1 cama k...",['1 cama','1 dormitorio','1 baño','1 cama king','Estacionamiento gratis','Wifi','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,1,1,1
821,53490471,"['4 camas', 'Monoambiente', '1 baño', '4 camas...",['4 camas','Monoambiente','1 baño','4 camas dobles','2 camas de una plaza','más','Wifi','Lavarropas','Cocina','Calefacción','Calificado con 4,8 de 5 en limpieza'],None,4,1,1
586,52169627,"['5 camas', '3 dormitorios', '2 baños', 'Estac...",['5 camas','3 dormitorios','2 baños','Estacionamiento gratis','Wifi','Calificado con 4,7 de 5 en limpieza'],None,None,None,None,None,None,5,3,2
363,842936153967499941,"['No hay evaluaciones', '1 cama', '1 dormitori...",['No hay evaluaciones','1 cama','1 dormitorio','1 baño','Wifi','No hay evaluaciones','No hay evaluaciones'],None,None,None,None,None,None,1,1,1


In [1055]:
df_caracteristicas.fillna({'cama': 1,'baño':1,'dormitorio':1},inplace= True)
mask_cama = df_caracteristicas['characteristics'].str.contains('cama')
mask_dormitorio = df_caracteristicas['characteristics'].str.contains('dormitorio')
mask_baño = df_caracteristicas['characteristics'].str.contains('baño')

df_caracteristicas.loc[~((mask_cama) & (mask_dormitorio) & (mask_baño))]['Columna1'].unique()

array([" 'Monoambiente'", " '1 dormitorio']", " '1 cama'", "5 baños'",
       " '1 cama doble'", " '6 camas'", " '1 dormitorio'",
       " '2 camas de una plaza'", None, " '1 baño privado'",
       "5 baños compartidos'", " '2 camas'", " '4 camas'",
       " '2 dormitorios'", " '2 camas dobles'",
       " 'Estacionamiento gratis'", " '1 sofá cama'", " 'Wifi'",
       " '1 cama queen'", " '4 camas de una plaza'", " '4 dormitorios'",
       " '3 dormitorios'"], dtype=object)